In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score, roc_auc_score
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder,  minmax_scale
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GroupShuffleSplit
import itertools
from torch.utils.data import DataLoader
import torch
from catboost import CatBoostRanker, Pool,  cv
import optuna
from optuna.integration import CatBoostPruningCallback
from catboost.utils import eval_metric
from optuna.pruners import SuccessiveHalvingPruner

In [2]:
# read the files changes path to where the data is stored
test = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\test_set_VU_DM.csv")
train = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\training_set_VU_DM.csv")
sample = pd.read_csv(r"C:\Users\youpz\Documents\Master\P5\Data mining techniques\Assignment2\data\dmt-2025-2nd-assignment\submission_sample.csv")

In [3]:
def missing_values_table(df):
    """Returns a DataFrame with missing counts and percent missing for each column."""
    n = len(df)
    missing_count   = df.isna().sum()
    missing_percent = 100 * missing_count / n
    missing_df = (
        pd.DataFrame({
            'missing_count':   missing_count,
            'missing_percent': missing_percent
        })
        .sort_values('missing_percent', ascending=False)
    )
    return missing_df

In [4]:
mv = missing_values_table(train)
print(mv.head(20))       # top 20 most‐missing columns
# Or to filter down to “lots” of missing, say >30%:
print(mv[mv['missing_percent'] > 30])

                           missing_count  missing_percent
comp1_rate_percent_diff          4863908        98.095353
comp6_rate_percent_diff          4862173        98.060362
comp1_rate                       4838417        97.581250
comp1_inv                        4828788        97.387053
comp4_rate_percent_diff          4827261        97.356256
gross_bookings_usd               4819957        97.208949
comp7_rate_percent_diff          4819832        97.206428
comp6_rate                       4718190        95.156511
visitor_hist_starrating          4706481        94.920364
visitor_hist_adr_usd             4705359        94.897735
comp6_inv                        4697371        94.736633
comp4_rate                       4650969        93.800797
comp7_rate                       4642999        93.640058
srch_query_affinity_score        4640941        93.598552
comp4_inv                        4614684        93.069001
comp7_inv                        4601925        92.811677
comp3_rate_per

In [27]:
########################## some beginning on the feature engineering
def missing_values_table(df):
    """Returns a DataFrame with missing counts and percent missing for each column."""
    n = len(df)
    missing_count   = df.isna().sum()
    missing_percent = 100 * missing_count / n
    missing_df = (
        pd.DataFrame({
            'missing_count':   missing_count,
            'missing_percent': missing_percent
        })
        .sort_values('missing_percent', ascending=False)
    )
    return missing_df
    
def preprocess_missing_and_competitors(train_df, test_df):
    # 1) Drop features with >93% missing or that leak the target
    drop_cols = [
        # competitor 1,4,6,7 are ~97–98% missing → too sparse to learn
        *[f'comp{i}_{t}' for i in [1,4,6,7] for t in ['rate','inv','rate_percent_diff']],
        'gross_bookings_usd'  # only in train, leaks booking price
    ]
    train_df.drop(columns=drop_cols, errors='ignore', inplace=True)
    test_df.drop(columns=drop_cols, errors='ignore', inplace=True)

    # 2) Impute & flag user history features
    #    Missing means “no prior purchases” → keep with sentinel + flag
    for df in (train_df, test_df):
        # visitor_hist_starrating
        df['hist_star_na'] = df['visitor_hist_starrating'].isna().astype(int)
        # fill with median starrating across users
        star_med = train_df['visitor_hist_starrating'].median()
        df['visitor_hist_starrating'] = df['visitor_hist_starrating'].fillna(star_med)

        # visitor_hist_adr_usd (avg USD spend)
        df['hist_adr_na'] = df['visitor_hist_adr_usd'].isna().astype(int)
        adr_med = train_df['visitor_hist_adr_usd'].median()
        df['visitor_hist_adr_usd'] = df['visitor_hist_adr_usd'].fillna(adr_med)

    # 3) Impute & flag affinity score
    #    Null means “hotel never seen” → fill with global minimum and flag
    affinity_min = train_df['srch_query_affinity_score'].min(skipna=True)
    for df in (train_df, test_df):
        df['affinity_na'] = df['srch_query_affinity_score'].isna().astype(int)
        df['srch_query_affinity_score'] = (
            df['srch_query_affinity_score']
            .fillna(affinity_min)
        )

    # 4) Keep & impute competitor 2,3,5,8 features (~50–90% missing)
    #    Null → “no data” sentinel (for categorical) or 0 (for percent diff), plus flag
    keep_comps = [2,3,5,8]
    for i in keep_comps:
        # availability flag
        inv_col = f'comp{i}_inv'
        flag_col = f'comp{i}_inv_na'
        for df in (train_df, test_df):
            df[flag_col] = df[inv_col].isna().astype(int)
            # fill null with 2 (new category: 0=no avail,1=avail,2=no data)
            df[inv_col] = df[inv_col].fillna(2).astype(int)

        # price‐compare flag
        rate_col = f'comp{i}_rate'
        rate_flag = f'comp{i}_rate_na'
        for df in (train_df, test_df):
            df[rate_flag] = df[rate_col].isna().astype(int)
            # fill null as “no data” = 2
            df[rate_col] = df[rate_col].fillna(2).astype(int)

        # percent_diff
        pdiff_col = f'comp{i}_rate_percent_diff'
        pdiff_flag = f'comp{i}_pdiff_na'
        for df in (train_df, test_df):
            df[pdiff_flag] = df[pdiff_col].isna().astype(int)
            # fill null as 0% diff (no info)
            df[pdiff_col] = df[pdiff_col].fillna(0.0)

    # 5) Bucket orig_destination_distance
    #    Missing → sentinel bucket + flag
    for df in (train_df, test_df):
        df['dist_na'] = df['orig_destination_distance'].isna().astype(int)
        df['orig_destination_distance'] = (
            df['orig_destination_distance'].fillna(-1)
        )
        # define bins (in km)
        bins = [-1, 0, 10, 50, 200, np.inf]
        labels = ['missing','0-10km','10-50km','50-200km','200km+']
        df['dist_bucket'] = pd.cut(
            df['orig_destination_distance'],
            bins=bins, labels=labels
        )

    return train_df, test_df

def create_base_features(df):
    """1) Parse datetime & basic price/historical features."""
    df = df.copy()
    # --- Date/time splits ---
    df['date_time']   = pd.to_datetime(df['date_time'])
    df['search_year'] = df['date_time'].dt.year
    df['search_month']= df['date_time'].dt.month
    df['search_day']  = df['date_time'].dt.day
    df['search_hour'] = df['date_time'].dt.hour

    # --- Price per night & hist price devation ---
    df['price_per_night'] = df['price_usd'] / df['srch_length_of_stay']
    df['price_vs_historical'] = df['price_usd'] - df['prop_log_historical_price']
    df['price_vs_historical'].fillna(0, inplace=True)
    return df

def add_destination_stats(train_df, test_df):
    """6) Dest‑level total searches & booking rate."""
    dest = (
        train_df
        .groupby('srch_destination_id')
        .agg(dest_searches=('srch_id','count'),
             dest_bookings=('booking_bool','sum'))
        .assign(dest_booking_rate=lambda x: x.dest_bookings / x.dest_searches)
        .reset_index()
    )
    # Reassign merge result back to each DataFrame
    train_df = train_df.merge(
        dest[['srch_destination_id','dest_searches','dest_booking_rate']],
        on='srch_destination_id', how='left'
    )
    test_df = test_df.merge(
        dest[['srch_destination_id','dest_searches','dest_booking_rate']],
        on='srch_destination_id', how='left'
    )
    return train_df, test_df

def add_within_search_features(df):
    """7) Z‑scores & deltas in each search group."""
    grp = df.groupby('srch_id')
    # price
    df['price_mean_srch'] = grp['price_usd'].transform('mean')
    df['price_std_srch']  = grp['price_usd'].transform('std').fillna(1)
    df['price_zscore']    = (df['price_usd'] - df['price_mean_srch']) / df['price_std_srch']
    # stars
    df['star_mean_srch']  = grp['prop_starrating'].transform('mean')
    df['star_delta_srch'] = df['prop_starrating'] - df['star_mean_srch']
    # user delta
    df['star_delta_user'] = df['prop_starrating'] - df['visitor_hist_starrating']
    # distance
    df['dist_mean_srch']  = grp['orig_destination_distance'].transform('mean')
    df['dist_std_srch']   = grp['orig_destination_distance'].transform('std').fillna(1)
    df['dist_zscore']     = (df['orig_destination_distance'] - df['dist_mean_srch']) / df['dist_std_srch']
    return df

def add_temporal_features(df):
    """8) Weekday/weekend & check‑in weekend flags."""
    # day‑of‑week for search
    df['search_dow'] = df['date_time'].dt.weekday  # 0=Mon…6=Sun
    df['is_search_weekend'] = df['search_dow'].isin([5,6]).astype(int)
    # approximate check‑in day
    checkin = df['date_time'] + pd.to_timedelta(df['srch_booking_window'], 'D')
    df['checkin_dow'] = checkin.dt.weekday
    df['is_checkin_weekend'] = df['checkin_dow'].isin([5,6]).astype(int)
    return df

def add_ranks(df):
    """9) Dense ranks of price, star & distance within each search."""
    df['price_rank'] = df.groupby('srch_id')['price_usd'].rank('dense', ascending=True)
    df['star_rank']  = df.groupby('srch_id')['prop_starrating'].rank('dense', ascending=False)
    df['dist_rank']  = df.groupby('srch_id')['orig_destination_distance'].rank('dense', ascending=True)
    return df

def one_hot_encode_columns(train_df, test_df, columns_to_encode):
    """
    Converts specified string columns in train and test DataFrames to one-hot encoded features.

    Args:
        train_df (pd.DataFrame): The training DataFrame.
        test_df (pd.DataFrame): The testing DataFrame.
        columns_to_encode (list): A list of column names (strings) to be one-hot encoded.

    Returns:
        tuple: A tuple containing the modified training and testing DataFrames with one-hot encoded columns.
    """
    train_processed = train_df.copy()
    test_processed = test_df.copy()

    for col in columns_to_encode:
        if col in train_processed.columns and col in test_processed.columns:
            # Get unique values from both train and test to ensure consistent encoding
            all_unique_values = pd.concat([train_processed[col], test_processed[col]]).unique()

            for value in all_unique_values:
                train_processed[f'{col}_{value}'] = (train_processed[col] == value).astype(int)
                test_processed[f'{col}_{value}'] = (test_processed[col] == value).astype(int)

            # Drop the original categorical column
            train_processed.drop(columns=[col], inplace=True)
            test_processed.drop(columns=[col], inplace=True)
        else:
            print(f"Warning: Column '{col}' not found in both train and test DataFrames. Skipping one-hot encoding for this column.")

    return train_processed, test_processed

def minmax(arr):
    return (arr - arr.min()) / (arr.max() - arr.min() + 1e-8)

def ensemble_predictions(pred1, pred2, weights):
    w1, w2 = weights
    return w1 * minmax(pred1) + w2 * minmax(pred2)

def preprocess_missing_and_competitors_fit(train_df):
    stats = {}
    # 1) Which cols to drop
    stats['drop_cols'] = [
        *[f'comp{i}_{t}' for i in [1,4,6,7] 
          for t in ['rate','inv','rate_percent_diff']],
        'gross_bookings_usd'
    ]
    # 2) Medians & mins for imputation
    stats['star_med']     = train_df['visitor_hist_starrating'].median()
    stats['adr_med']      = train_df['visitor_hist_adr_usd'].median()
    stats['affinity_min'] = train_df['srch_query_affinity_score'].min(skipna=True)
    return stats

def preprocess_missing_and_competitors_transform(df, stats):
    df = df.copy()
    df.drop(columns=stats['drop_cols'], errors='ignore', inplace=True)

    # visitor history
    df['hist_star_na'] = df['visitor_hist_starrating'].isna().astype(int)
    df['visitor_hist_starrating'] = (
        df['visitor_hist_starrating']
          .fillna(stats['star_med'])
    )
    df['hist_adr_na'] = df['visitor_hist_adr_usd'].isna().astype(int)
    df['visitor_hist_adr_usd'] = (
        df['visitor_hist_adr_usd']
          .fillna(stats['adr_med'])
    )

    # affinity
    df['affinity_na'] = df['srch_query_affinity_score'].isna().astype(int)
    df['srch_query_affinity_score'] = (
        df['srch_query_affinity_score']
          .fillna(stats['affinity_min'])
    )

    # keep comps 2,3,5,8
    for i in [2,3,5,8]:
        for col, fill in [
            (f'comp{i}_inv',               2),
            (f'comp{i}_rate',              2),
            (f'comp{i}_rate_percent_diff', 0.0),
        ]:
            df[f'{col}_na'] = df[col].isna().astype(int)
            df[col] = df[col].fillna(fill)

    # distance bucket
    df['dist_na'] = df['orig_destination_distance'].isna().astype(int)
    df['orig_destination_distance'] = (
        df['orig_destination_distance'].fillna(-1)
    )
    bins  = [-1,0,10,50,200,np.inf]
    lbls  = ['missing','0-10km','10-50km','50-200km','200km+']
    df['dist_bucket'] = pd.cut(
        df['orig_destination_distance'], bins=bins, labels=lbls
    )
    return df

def add_destination_stats_fit(train_df):
    dest = (
        train_df
        .groupby('srch_destination_id')
        .agg(dest_searches=('srch_id','count'),
             dest_bookings=('booking_bool','sum'))
    )
    dest['dest_booking_rate'] = dest['dest_bookings'] / dest['dest_searches']
    return dest

def add_destination_stats_transform(df, dest_stats):
    df = df.copy()
    return df.merge(
        dest_stats[['dest_searches','dest_booking_rate']],
        on='srch_destination_id', how='left'
    )
def run_pipeline_on(train_df, other_df, steps):
    df1, df2 = train_df.copy(), other_df.copy()
    for fn in steps:
        df1, df2 = fn(df1, df2)
    return df1, df2

In [6]:
# your FE pipeline, exactly as before
steps = [
    preprocess_missing_and_competitors,
    add_destination_stats,
    lambda tr, te: one_hot_encode_columns(tr, te, ['dist_bucket']),
    lambda tr, te: (create_base_features(tr),     create_base_features(te)),
    lambda tr, te: (add_within_search_features(tr), add_within_search_features(te)),
    lambda tr, te: (add_temporal_features(tr),    add_temporal_features(te)),
    lambda tr, te: (add_ranks(tr),                add_ranks(te)),
]

In [7]:
mv = missing_values_table(train)
print(mv.head(20))       # top 20 most‐missing columns
# Or to filter down to “lots” of missing, say >30%:
print(mv[mv['missing_percent'] > 30])

                           missing_count  missing_percent
comp1_rate_percent_diff          4863908        98.095353
comp6_rate_percent_diff          4862173        98.060362
comp1_rate                       4838417        97.581250
comp1_inv                        4828788        97.387053
comp4_rate_percent_diff          4827261        97.356256
gross_bookings_usd               4819957        97.208949
comp7_rate_percent_diff          4819832        97.206428
comp6_rate                       4718190        95.156511
visitor_hist_starrating          4706481        94.920364
visitor_hist_adr_usd             4705359        94.897735
comp6_inv                        4697371        94.736633
comp4_rate                       4650969        93.800797
comp7_rate                       4642999        93.640058
srch_query_affinity_score        4640941        93.598552
comp4_inv                        4614684        93.069001
comp7_inv                        4601925        92.811677
comp3_rate_per

In [33]:
#0) define target
y = train['booking_bool'] * 5 + train['click_bool']

# 1) Split out search-IDs into train vs. valid (never touching test until after FE)
train_ids, valid_ids = train_test_split(
    train['srch_id'].unique(),
    test_size=0.2,
    random_state=22
)
train_raw = train[ train['srch_id'].isin(train_ids) ].reset_index(drop=True)
valid_raw = train[ train['srch_id'].isin(valid_ids) ].reset_index(drop=True)
test_raw  = test.copy()


# 3) “Fit” FE on train_raw + test_raw
train_feat, test_feat = run_pipeline_on(train_raw, test_raw, steps)

# 4) “Apply” identical FE to valid_raw by feeding in the already-fitted train_feat
_, valid_feat = run_pipeline_on(train_feat, valid_raw, steps)

# 5) Select feature columns
drop_cols = ['date_time','gross_bookings_usd','position',
             'click_bool','booking_bool','srch_id','prop_id']
features = [c for c in train_feat.columns if c not in drop_cols]

X        = train_feat[features]
X_va_all = valid_feat[features]
X_test   = test_feat[features]

# 6) Replace infinities and NaNs using only TRAIN medians
for df in (X, X_va_all, X_test):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
medians = X.median()
X     .fillna(medians, inplace=True)
X_va_all.fillna(medians, inplace=True)
X_test .fillna(medians, inplace=True)

# 7) Scale
scaler = StandardScaler()
X_tr   = scaler.fit_transform(X)                       # train only
X_va   = scaler.transform(X_va_all)                    # valid
X_full = scaler.transform(pd.concat([X, X_va_all]))    # full
X_test = scaler.transform(X_test)                      # test

# 8) Build y arrays
y_tr   = (train_raw['booking_bool'] * 5 + train_raw['click_bool']).values
y_va   = (valid_raw['booking_bool'] * 5 + valid_raw['click_bool']).values
y_full = np.concatenate([y_tr, y_va])

# 9) Session‐IDs and group‐sizes
id_tr   = train_raw['srch_id'].values
id_va   = valid_raw['srch_id'].values
id_full = np.concatenate([id_tr, id_va])

grp_tr   = train_raw['srch_id'].value_counts(sort=False).values
grp_va   = valid_raw['srch_id'].value_counts(sort=False).values
grp_full = np.concatenate([grp_tr, grp_va])

# 10) Make LightGBM & CatBoost datasets
lgb_train = lgb.Dataset(X_tr,   label=y_tr,   group=grp_tr)
lgb_val   = lgb.Dataset(X_va,   label=y_va,   group=grp_va)
lgb_full  = lgb.Dataset(X_full, label=y_full, group=grp_full)

train_pool = Pool(X_tr,   label=y_tr,   group_id=id_tr)
val_pool   = Pool(X_va,   label=y_va,   group_id=id_va)
full_pool  = Pool(X_full, label=y_full, group_id=id_full)


C:\Users\youpz\AppData\Local\Temp\ipykernel_6572\227212721.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price_vs_historical'].fillna(0, inplace=True)
C:\Users\youpz\AppData\Local\Temp\ipykernel_6572\227212721.py:108: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [34]:
# 1) Define your search space

param_dist = {
  'learning_rate':    [0.01,0.015,0.02,0.025],
  'num_leaves':       [128,192,256,320],
  'min_data_in_leaf': [5,10,15,20],
  'feature_fraction': [0.5,0.6,0.7],
  'bagging_fraction': [0.8,1.0],
  'bagging_freq':     [0,5],
  'lambda_l1':        [0,0.1,0.5],
  'lambda_l2':        [0,0.1,0.5],
  'min_gain_to_split':[0,0.1,0.2]
}

n_iter = 30
best_score, score = 0,0
best_params,params = None,0

In [35]:
# 1) Your random search stays the same…
for i in range(1, n_iter + 1):
    print(i)
    params = {
        'objective':         'lambdarank',
        'metric':            'ndcg',
        'ndcg_eval_at':      [5],
        'verbose':           1,
        'feature_pre_filter': False,      # ← disable the one-shot pre-filter
    }
    # 2) then overlay your random hyper‐params
    params.update({k: np.random.choice(v) for k, v in param_dist.items()})
    print(params)
    model = lgb.train(
        params,
        train_set=lgb_train,
        num_boost_round=600,
        valid_sets=lgb_val,
        valid_names=['valid'],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    
    score = model.best_score['valid']['ndcg@5']
    print(f"    → ndcg@5 = {score:.4f}, rounds = {model.best_iteration}")
    
    if score > best_score:
        best_score_ = score
        best_params = params.copy()
        best_iter =  model.best_iteration

1
{'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [5], 'verbose': 1, 'feature_pre_filter': False, 'learning_rate': 0.015, 'num_leaves': 320, 'min_data_in_leaf': 15, 'feature_fraction': 0.6, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'lambda_l1': 0.5, 'lambda_l2': 0.0, 'min_gain_to_split': 0.2}
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.231763 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6874
[LightGBM] [Info] Number of data points in the train set: 3968534, number of used features: 79
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[599]	valid's ndcg@5: 0.39209
    → ndcg@5 = 0.3921, rounds = 599
2
{'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [5], 'verbose': 1, 'feature_pre_filter': False, 'learning_rate': 0.015, 'num_leaves': 128, 'min_

In [36]:
print(best_score, best_params, best_iter)

0 {'objective': 'lambdarank', 'metric': 'ndcg', 'ndcg_eval_at': [5], 'verbose': 1, 'feature_pre_filter': False, 'learning_rate': 0.02, 'num_leaves': 192, 'min_data_in_leaf': 20, 'feature_fraction': 0.6, 'bagging_fraction': 0.8, 'bagging_freq': 5, 'lambda_l1': 0.0, 'lambda_l2': 0.1, 'min_gain_to_split': 0.0} 511


In [37]:
# 3) Retrain on all data
model_lgb_full = lgb.train(
    best_params,
    lgb_full,
    num_boost_round=best_iter # can still be ajdusted to best best_iter
)

# 4) Predict on test
lgb_test_preds = model_lgb_full.predict(X_test)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.423288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6892
[LightGBM] [Info] Number of data points in the train set: 4958347, number of used features: 79


In [56]:
def objective(trial):
    params_cat = {
        'loss_function': 'YetiRank',
        'eval_metric': 'NDCG:top=5',
        'random_seed': 5,
        'od_type': 'Iter',
        'od_wait': 20,
        'learning_rate': trial.suggest_float("learning_rate", 0.01, 0.1, log=True),
        'depth': trial.suggest_int("depth", 6, 12),
        'l2_leaf_reg': trial.suggest_float("l2_leaf_reg", 1e-3, 10.0, log=True),
        'random_strength': trial.suggest_float("random_strength", 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float("bagging_temperature", 0.0, 1.0),
        'border_count': trial.suggest_int("border_count", 32, 255),
        'iterations': 600,
        'use_best_model': True,
        'verbose': 100,
    }

    model_cat = CatBoostRanker(**params_cat)

    # Pruner callback for PFound
    pruning_callback = CatBoostPruningCallback(trial, 'NDCG:top=5;type=Base')


    model_cat.fit(
        train_pool,
        eval_set=val_pool,
        early_stopping_rounds=50,
        callbacks=[pruning_callback],
    )

    # Manually trigger pruning
    pruning_callback.check_pruned()

     # 5) Grab the best‐iteration score
    hist = model_cat.get_evals_result()['validation']['NDCG:top=5;type=Base']
    return max(hist) 


study = optuna.create_study(
    direction="maximize",
    study_name="catboost_yetirank",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
)
study.optimize(objective, n_trials=60, timeout=7200)

print(f"Best PFound: {study.best_trial.value}")
print("Best hyperparameters:")
for k, v in study.best_trial.params.items():
    print(f"  {k}: {v}")

[I 2025-05-14 23:04:59,547] A new study created in memory with name: catboost_yetirank
C:\Users\youpz\AppData\Local\Temp\ipykernel_6572\1916531565.py:22: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, 'NDCG:top=5;type=Base')


0:	test: 0.2737375	best: 0.2737375 (0)	total: 2.06s	remaining: 20m 35s
100:	test: 0.3656543	best: 0.3656543 (100)	total: 3m 25s	remaining: 16m 57s
200:	test: 0.3729270	best: 0.3729607 (195)	total: 6m 29s	remaining: 12m 52s
300:	test: 0.3767713	best: 0.3767713 (300)	total: 8m 45s	remaining: 8m 41s
400:	test: 0.3786513	best: 0.3786666 (399)	total: 11m 2s	remaining: 5m 28s
500:	test: 0.3802330	best: 0.3802714 (495)	total: 13m 17s	remaining: 2m 37s


[I 2025-05-14 23:20:35,388] Trial 0 finished with value: 0.38174025809791046 and parameters: {'learning_rate': 0.04342808426146904, 'depth': 7, 'l2_leaf_reg': 0.023500321607835283, 'random_strength': 2.1941741481158905, 'bagging_temperature': 0.6082278641126276, 'border_count': 101}. Best is trial 0 with value: 0.38174025809791046.


599:	test: 0.3817105	best: 0.3817403 (592)	total: 15m 32s	remaining: 0us

bestTest = 0.3817402581
bestIteration = 592

Shrink model to first 593 iterations.
0:	test: 0.2740788	best: 0.2740788 (0)	total: 1.54s	remaining: 15m 23s
100:	test: 0.3618343	best: 0.3618343 (100)	total: 2m 23s	remaining: 11m 47s
200:	test: 0.3694227	best: 0.3694398 (196)	total: 4m 40s	remaining: 9m 17s
300:	test: 0.3736560	best: 0.3736560 (300)	total: 7m	remaining: 6m 57s
400:	test: 0.3763994	best: 0.3763994 (400)	total: 9m 15s	remaining: 4m 35s
500:	test: 0.3778578	best: 0.3780190 (498)	total: 11m 32s	remaining: 2m 16s


[I 2025-05-14 23:34:23,522] Trial 1 finished with value: 0.37964035216889236 and parameters: {'learning_rate': 0.030692101658412366, 'depth': 7, 'l2_leaf_reg': 0.00653660019939535, 'random_strength': 0.026125063661446377, 'bagging_temperature': 0.9244255378586388, 'border_count': 222}. Best is trial 0 with value: 0.38174025809791046.


...


bestTest = 0.289164347
bestIteration = 5

Shrink model to first 6 iterations.
0:	test: 0.2942665	best: 0.2942665 (0)	total: 2.32s	remaining: 23m 8s


[I 2025-05-15 01:03:26,850] Trial 8 pruned. Trial was pruned at iteration 5.



bestTest = 0.3232359623
bestIteration = 5

Shrink model to first 6 iterations.
0:	test: 0.2869184	best: 0.2869184 (0)	total: 2.44s	remaining: 24m 23s
100:	test: 0.3735251	best: 0.3735251 (100)	total: 3m 59s	remaining: 19m 43s
200:	test: 0.3782880	best: 0.3782880 (200)	total: 7m 52s	remaining: 15m 38s
300:	test: 0.3828303	best: 0.3829524 (299)	total: 11m 47s	remaining: 11m 42s
400:	test: 0.3843198	best: 0.3844697 (391)	total: 15m 45s	remaining: 7m 49s
500:	test: 0.3854783	best: 0.3857583 (498)	total: 19m 42s	remaining: 3m 53s


[I 2025-05-15 01:26:50,640] Trial 9 finished with value: 0.3862178421739308 and parameters: {'learning_rate': 0.06357340379483874, 'depth': 12, 'l2_leaf_reg': 0.007661024957395861, 'random_strength': 7.686573664667392, 'bagging_temperature': 0.2043937457253162, 'border_count': 83}. Best is trial 4 with value: 0.38848251696097624.


Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.3862178422
bestIteration = 542

Shrink model to first 543 iterations.
Best PFound: 0.38848251696097624
Best hyperparameters:
  learning_rate: 0.09515705698611983
  depth: 9
  l2_leaf_reg: 0.6947324643801653
  random_strength: 2.608675094800671
  bagging_temperature: 0.6429616492483743
  border_count: 250


In [13]:
# 1) Grab your best config & best score from Optuna
best_params_cat = study.best_trial.params
best_ndcg5      = study.best_trial.value

print(f"Best validation NDCG@5 (Optuna): {best_ndcg5:.4f}")
print("Best hyperparameters:")
for key, value in best_params_cat.items():
    print(f"  {key}: {value}")

# 2) Re-instantiate your CatBoostRanker with a very large `iterations` …
final_params_cat = {
    **best_params_cat,
    'loss_function':   'YetiRank',
    'eval_metric':     'NDCG:top=5',
    'random_seed':     22,
    'use_best_model':  True,
    'verbose':         100,
    'iterations':      2000,            # start with something big …
}
full_model = CatBoostRanker(**final_params_cat)

# 3) Fit on the full training+validation Pool, using early stopping
#    to discover the optimum number of trees
full_model.fit(
    full_pool,
    early_stopping_rounds=100,
)

# 4) Pull out the “best iteration” we actually used
best_iter_full = full_model.get_best_iteration()
print(f"➡️  Full‐data training stopped at iteration {best_iter_full}")

# 5) Now **freeze** your model to exactly that number of trees:
#    re-build with iterations=best_iter_full (no early stopping)
final_params_cat['iterations'] = best_iter_full
final_model = CatBoostRanker(**final_params_cat)
final_model.fit(full_pool, verbose=False)

# 6) Predict on your test set
cat_test_preds = final_model.predict(
    X_test, 
    prediction_type='RawFormulaVal'  # this is the default return for a ranker
)

# 7) (Optional) If you want to blend with LightGBM…
# ensemble_preds = 0.4*minmax(cat_test_preds) + 0.6*minmax(lgb_test_preds)


NameError: name 'study' is not defined

In [9]:
# 4) Predict on test set
cat_test_preds = final_cat.predict(X_test)


NameError: name 'final_cat' is not defined

In [59]:
# 2) Put them into your submission

#can use cat_test_pred or lgb_test_preds

sample['score'] = lgb_test_preds
submission = (
    sample
    .sort_values(['srch_id', 'score'], ascending=[True, False])
    [['srch_id', 'prop_id']]
)

# 3) Write to CSV
submission.to_csv('submission.csv', index=False)
print("submission.csv written using lgb only!") 

submission.csv written using lgb only!


In [1]:
# 1. Predict raw scores for the validation set
cat_val_preds = final_cat.predict(X_va)
lgb_val_preds = model_lgb_full.predict(X_va)

# 2. Compute per-session NDCG@5 manually
ndcgs_cat = []
ndcgs_lgb = []

for q in np.unique(id_va):
    idx = np.where(id_va == q)[0]
    if len(idx) > 1:  # skip sessions with only 1 result
        true_rels = y_va[idx]
        ndcgs_cat.append(ndcg_score([true_rels], [cat_val_preds[idx]], k=5))
        ndcgs_lgb.append(ndcg_score([true_rels], [lgb_val_preds[idx]], k=5))

# 3. Average NDCG@5 for both models
catboost_ndcg5 = np.mean(ndcgs_cat)
lgb_ndcg5 = np.mean(ndcgs_lgb)

print(f"\nCatBoost Val NDCG@5: {catboost_ndcg5:.4f}")
print(f"\nLightGBM Val NDCG@5: {lgb_ndcg5:.4f}")


NameError: name 'final_cat' is not defined

In [ ]:


# 1) Scale just your two tree-based model preds on validation
cb_s  = minmax(cat_val_preds)   # your CatBoost val preds
lgb_s = minmax(lgb_val_preds)   # your LightGBM val preds

# 2) Sweep over blends of just CB + LGB
best, best_w = 0, None
for w_cb in np.linspace(0, 1, 11):
    w_lgb = 1 - w_cb
    sc = w_cb * cb_s + w_lgb * lgb_s

    # compute NDCG@5 on val
    ndcgs = []
    for q in np.unique(id_va):
        idx = np.where(id_va == q)[0]
        if len(idx) > 1:
            true = y_va[idx]
            ndcgs.append(ndcg_score([true], [sc[idx]], k=5))
    mean_ndcg = np.mean(ndcgs)

    print(f"w_cb={w_cb:.1f}, w_lgb={w_lgb:.1f} → Val NDCG@5: {mean_ndcg:.4f}")
    if mean_ndcg > best:
        best, best_w = mean_ndcg, (w_cb, w_lgb)

print(f"\n Best blend (CB+LGB): w_cb={best_w[0]:.2f}, w_lgb={best_w[1]:.2f} → NDCG@5={best:.4f}")


In [ ]:
# --- Ensemble the test predictions with your optimal blend ---
best_w = [0.5,0.5]

ensemble_final_preds = ensemble_predictions(
    cat_test_preds,
    lgb_test_preds,
    weights=[best_w[0], best_w[1]]
)

# Put them into submission
sample['score'] = ensemble_final_preds
submission = (
    sample
    .sort_values(['srch_id','score'], ascending=[True, False])
    [['srch_id','prop_id']]
)
submission.to_csv('submission.csv', index=False)
print(f"Submission.csv written using w_cb={best_w[0]:.2f}, w_lgb={best_w[1]:.2f}!") 


In [ ]:
"""
# --- 1) ListNet loss (unchanged) ---
def listnet_loss(scores, labels, group_ids):
    loss, count = 0.0, 0
    for q in np.unique(group_ids):
        idx = np.where(group_ids == q)[0]
        if len(idx) < 2:
            continue
        s_q, y_q = scores[idx], labels[idx].float()
        P, P_hat = torch.softmax(y_q, 0), torch.softmax(s_q, 0)
        loss += -torch.sum(P * torch.log(P_hat + 1e-8))
        count += 1
    return loss / max(count, 1)

param_grid = {
    'lr':           [1e-3, 2e-3],
    'batch_size':   [512, 1024],
    'dropout':      [0.1, 0.2],
    'weight_decay': [0.0, 1e-4]
}

best = {'ndcg': 0.0, 'cfg': None}

for lr, bs, drop, wd in itertools.product(*param_grid.values()):
    tr_loader = DataLoader(ExpediaDataset(X_tr, y_tr.values),
                           batch_size=bs, shuffle=True)
    va_loader = DataLoader(ExpediaDataset(X_va, y_va.values),
                           batch_size=bs, shuffle=False)

    mdl = DeepRecommender(X_tr.shape[1]).to(device)
    for m in mdl.modules():
        if isinstance(m, nn.Dropout):
            m.p = drop

    opt = torch.optim.Adam(
        mdl.parameters(),
        lr=lr,
        weight_decay=wd
    )
    sched = torch.optim.lr_scheduler.OneCycleLR(
        opt,
        max_lr=lr * 10,
        steps_per_epoch=len(tr_loader),
        epochs=10
    )

    best_ndcg, stale = 0.0, 0
    for epoch in range(1, 8):
        # — train —
        mdl.train()
        for Xb, yb in tr_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            opt.zero_grad()
            loss = listnet_loss(mdl(Xb), yb, id_tr[:len(Xb)])
            loss.backward()
            opt.step()

        # — validate —
        mdl.eval()
        preds = []
        with torch.no_grad():
            for Xb, _ in va_loader:
                preds.extend(mdl(Xb.to(device)).cpu().numpy())
        preds = np.array(preds)

        # compute mean NDCG@5 using the same y_va array
        ndcgs = []
        for q in np.unique(id_va):
            idx = np.where(id_va == q)[0]
            if len(idx) > 1:
                true = y_va.values[idx]
                score = preds[idx]
                ndcgs.append(ndcg_score([true], [score], k=5))
        mean_ndcg = np.mean(ndcgs)
        sched.step(mean_ndcg)

        if mean_ndcg > best_ndcg + 1e-4:
            best_ndcg, stale = mean_ndcg, 0
        else:
            stale += 1
        if stale >= 5:
            break

    print(f"cfg lr={lr}, bs={bs}, drop={drop} → Val NDCG@5: {best_ndcg:.4f}")
    if best_ndcg > best['ndcg']:
        best.update({'ndcg': best_ndcg, 'cfg': (lr, bs, drop)})

print("🏆 Best config:", best)
"""




In [ ]:
"""# unpack your best cfg and tweak
lr, bs, drop, wd = 1e-3, 128, 0.1, 1e-5

# rebuild full-training loader
full_loader = DataLoader(
    ExpediaDatasetGrouped(X_full, y_full, id_full),
    batch_size=bs, shuffle=True
)

# 1) Slightly larger network at the top
class BiggerRecommender(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512), nn.ReLU(), nn.BatchNorm1d(512), nn.Dropout(drop),
            nn.Linear(512, 256), nn.ReLU(), nn.BatchNorm1d(256), nn.Dropout(drop),
            nn.Linear(256, 128), nn.ReLU(), nn.BatchNorm1d(128), nn.Dropout(drop),
            nn.Linear(128, 64),  nn.ReLU(), nn.Dropout(drop),
            nn.Linear(64, 1)
        )
    def forward(self, x):
        return self.net(x).squeeze(1)

final_nn = BiggerRecommender(X_full.shape[1]).to(device)

# 2) Optimizer with very light weight decay
opt = torch.optim.Adam(final_nn.parameters(), lr=lr, weight_decay=wd)

# 3) Simpler stepLR scheduler (cuts LR by 0.5 every 10 epochs)
sched = torch.optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.5)

# 4) Train for 30 epochs with gradient clipping
for epoch in range(1, 31):
    final_nn.train()
    total_loss = 0.0
    for Xb, yb, gb in full_loader:
        Xb, yb = Xb.to(device), yb.to(device)

        opt.zero_grad()
        logits = final_nn(Xb)
        loss   = listnet_loss(logits, yb, gb)  
        loss.backward()
        torch.nn.utils.clip_grad_norm_(final_nn.parameters(), 5.0)
        opt.step()

        total_loss += loss.item() * Xb.size(0)

    sched.step()
    avg_loss = total_loss / len(full_loader.dataset)
    print(f"Epoch {epoch:02d}/30 — Avg ListNet Loss: {avg_loss:.4f} — LR: {opt.param_groups[0]['lr']:.1e}")
"""

In [ ]:
"""# --- 4) Produce final test preds ---
test_loader = DataLoader(
    ExpediaDataset(X_test, np.zeros(len(X_test))),
    batch_size=bs,
    shuffle=False
)

nn_test_preds = []
final_nn.eval()
with torch.no_grad():
    for Xb, _ in test_loader:           # <-- unpack both X and dummy y
        Xb = Xb.to(device)
        nn_test_preds.extend(
            torch.sigmoid(final_nn(Xb))
                 .cpu()
                 .numpy()
        )
nn_test_preds = np.array(nn_test_preds)

print("NN preds:", nn_test_preds.min(), nn_test_preds.max())
"""

In [ ]:
"""

# 1) Get NN probabilities on the validation fold
nn_val_probs = []
final_nn.eval()
with torch.no_grad():
    for Xb, _ in va_loader:
        nn_val_probs.extend(
            torch.sigmoid(final_nn(Xb.to(device)))
            .cpu()
            .numpy()
        )
nn_val_probs = np.array(nn_val_probs)  # shape = (n_val,)

# 2) Compute per-session NDCG@5
ndcgs = []
for q in np.unique(id_va):
    idx = np.where(id_va == q)[0]
    if len(idx) > 1:
        true_rels = y_val_nn[idx]
        scores    = nn_val_probs[idx]
        ndcgs.append(ndcg_score([true_rels], [scores], k=5))

nn_val_ndcg5 = np.mean(ndcgs)
print(f"NN  Val NDCG@5: {nn_val_ndcg5:.4f}")
"""

In [ ]:
"""# --- Ensemble the predictions ---
ensemble_final_preds = ensemble_predictions(nn_test_preds, model_lgb_pred, weights=[0.5, 0.5]) # Adjust weights

# Predict relevance scores for each test row
preds_ens = ensemble_final_preds

# Insert those scores into the sample submission DataFrame
sample['score'] = preds_ens

# Sort by search session (ascending) and score (descending)
#   so that for each srch_id, the most relevant prop_id comes first
submission = sample.sort_values(
    ['srch_id', 'score'],
    ascending=[True, False]
)

# keep only the required columns and write to CSV
#   Kaggle expects: srch_id, prop_id (in ranked order)
submission[['srch_id', 'prop_id']].to_csv(
    'submission.csv',
    index=False
)
print("Submission.csv adjusted with new scores!")"""

In [ ]:
"""
# 1) LightGBM val set predictions
# Replace `model_lgb` with whatever variable you named your trained LightGBM model
val_preds_lgb = model_lgb.predict(X_va)   # shape = (n_val,)

# 2) NN val set predictions
final_nn.eval()
nn_val_preds = []
with torch.no_grad():
    for Xb, _ in va_loader:  # va_loader from your NN split on X_va/y_va
        nn_val_preds.extend(
            torch.sigmoid(final_nn(Xb.to(device))).cpu().numpy()
        )
nn_val_preds = np.array(nn_val_preds)      # shape = (n_val,)

# 3) Blend them
w_nn, w_lgb = 0.4, 0.6
ensemble_val = w_nn * nn_val_preds + w_lgb * val_preds_lgb


ndcgs = []
for q in np.unique(id_va):
    idx = np.where(id_va == q)[0]
    if len(idx) > 1:
        true_rel  = y_val_nn[idx]    # your val labels array
        score_rel = ensemble_val[idx]
        ndcgs.append(ndcg_score([true_rel], [score_rel], k=5))

mean_ndcg5 = np.mean(ndcgs)
print(f"Ensembled Val NDCG@5: {mean_ndcg5:.4f}")
"""

In [ ]:
"""

# 1) Min–max scale each prediction array into [0,1]
def minmax(arr):
    return (arr - arr.min()) / (arr.max() - arr.min() + 1e-8)

nn_scaled  = minmax(nn_test_preds)
lgb_scaled = minmax(model_lgb_pred)   # or whatever your LGB test‐preds variable is

# 2) Do the two blends
ens1 = ensemble_predictions(nn_scaled, lgb_scaled, [0.5, 0.5])
ens2 = ensemble_predictions(nn_scaled, lgb_scaled, [0.0, 1.0])

# 3) Compare their sorted‐order permutations
order1 = np.argsort(ens1)
order2 = np.argsort(ens2)

# 4) Compute fraction of positions that differ
fraction_changed = np.mean(order1 != order2)
print(f"Fraction of test‐rows whose position changes: {fraction_changed:.4%}")
"""

In [ ]:
"""

# 0) Scale both NN & LGB validation predictions into [0,1]
nn_scaled_val  = minmax_scale(nn_val_preds)      # shape = (n_val,)
lgb_scaled_val = minmax_scale(val_preds_lgb)     # shape = (n_val,)

best_w, best_score = None, 0.0
for w_nn in np.linspace(0, 1, 11):
    w_lgb = 1 - w_nn

    # blended on the same [0,1] scale
    blended = w_nn * nn_scaled_val + w_lgb * lgb_scaled_val

    # compute NDCG@5 per session
    ndcgs = []
    for q in np.unique(id_va):
        idx = np.where(id_va == q)[0]
        if len(idx) > 1:
            true   = y_val_nn[idx]
            scores = blended[idx]
            ndcgs.append(ndcg_score([true], [scores], k=5))
    mean_ndcg = np.mean(ndcgs)

    print(f"w_nn={w_nn:.1f}, w_lgb={w_lgb:.1f} → Val NDCG@5: {mean_ndcg:.4f}")
    if mean_ndcg > best_score:
        best_score, best_w = mean_ndcg, w_nn

print(f"\n🏆 Best blend: w_nn={best_w:.2f}, w_lgb={1-best_w:.2f} → NDCG@5={best_score:.4f}")
"""

In [ ]:
"""df = sample[['srch_id']].copy()
df['nn_rank']  = pd.DataFrame({'score': nn_test_preds,  'srch': sample['srch_id']}) \
                    .groupby('srch')['score'] \
                    .rank(method='dense', ascending=False)
df['lgb_rank'] = pd.DataFrame({'score': model_lgb_pred,'srch': sample['srch_id']}) \
                    .groupby('srch')['score'] \
                    .rank(method='dense', ascending=False)

# weighted rank
w_nn, w_lgb = best_w, (1-best_w)
df['ensemble_rank'] = w_nn * df['nn_rank'] + w_lgb * df['lgb_rank']

# use that to sort
submission = sample.assign(_rank=df['ensemble_rank']) \
    .sort_values(['srch_id','_rank'], ascending=[True,True]) \
    [['srch_id','prop_id']]

# keep only the required columns and write to CSV
#   Kaggle expects: srch_id, prop_id (in ranked order)
submission[['srch_id', 'prop_id']].to_csv(
    'submission.csv',
    index=False
)
print("Submission.csv adjusted with new scores!")"""

In [ ]:
"""df_sub = sample[['srch_id']].copy()
df_sub['nn_rank']  = pd.Series(nn_test_preds).groupby(sample['srch_id']).rank("dense", ascending=False)
df_sub['lgb_rank'] = pd.Series(model_lgb_pred).groupby(sample['srch_id']).rank("dense", ascending=False)

# weighted rank blend
df_sub['ensemble_rank'] = 0.2*df_sub['nn_rank'] + 0.8*df_sub['lgb_rank']

submission = (
    sample.assign(_rank=df_sub['ensemble_rank'])
          .sort_values(['srch_id','_rank'], ascending=[True,True])
          [['srch_id','prop_id']]
)
# keep only the required columns and write to CSV
#   Kaggle expects: srch_id, prop_id (in ranked order)
submission[['srch_id', 'prop_id']].to_csv(
    'submission.csv',
    index=False
)
print("Submission.csv adjusted with new scores!")"""